# Initialization

## Import Library

In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
import random
import keras
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

Using Theano backend.
ERROR (theano.sandbox.cuda): Failed to compile cuda_ndarray.cu: dlopen(/Users/demiguo/.theano/compiledir_Darwin-13.4.0-x86_64-i386-64bit-i386-2.7.12-64/cuda_ndarray/cuda_ndarray.so, 2): Symbol not found: _OSAtomicDecrement32Barrier
  Referenced from: /usr/local/cuda/lib/libcublas.8.0.dylib
  Expected in: /usr/lib/libSystem.B.dylib
 in /usr/local/cuda/lib/libcublas.8.0.dylib


## Retrieve Data

In [2]:
filename = "../../data/labeled_data.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# [char_to_int[char] for char in str]

In [3]:
data = []
labels = []
with open(filename) as f:
    for line in f:
        s_len = len(line)
        block = 1000  # ignore
        cur = 0
        while (cur < s_len - 3):
            if (cur + block < s_len - 3):
                # only do 500 for now
                data.append(line[cur:cur+block].lower())
                labels.append(line[-2])
                cur += block
            else:
                data.append(line[cur:-3].lower())
                labels.append(line[-2])
                cur = s_len - 3
 

## Clean Data

In [4]:
# Now, let's clean punctuations 
for i, sentence in enumerate(data):
    for char in ".,&%?'":
        data[i] = data[i].replace(char, ' ')

## Split Data

In [5]:
n = len(data)  # data size
X_train = []
X_test = []
y_train = []
y_test = []
for i in range(n):
    word_vec = [char_to_int[char] for char in data[i]]
    label = labels[i]
    if random.randint(1,4) <= 3:
        # train
        X_train.append(word_vec) 
        y_train.append(label)
    else:
        # test
        X_test.append(word_vec)
        y_test.append(label)

## Pad Data

In [6]:
max_speech_length = max([len(sentence) for sentence in data])
X_train = sequence.pad_sequences(X_train, maxlen=max_speech_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_speech_length)

# Training

## Create the model

In [7]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(n, embedding_vecor_length, input_length=max_speech_length))
model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_length=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, nb_epoch=20, batch_size=64)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 656, 32)       517280      embedding_input_1[0][0]          
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 656, 32)       3104        embedding_1[0][0]                
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)    (None, 328, 32)       0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 100)           53200       maxpooling1d_1[0][0]             
___________________________________________________________________________________________

## Final evaluation of the model

In [8]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.89%
